In [73]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import seaborn as sns
from scipy import stats
import sys
from sklearn.preprocessing import normalize

In [74]:
sys.path.insert(0, '/Users/cynthiachen/Downloads/Internship2019/degron_mutation/deepDegron')
from train_nn import *

In [167]:
# Load pre-trained neural network models from pickle file
model_bag_of_words = pickle.load(open( "../models/neural_network_bag_of_words_v2.pickle", "rb" ))
model_pos_specific = pickle.load(open( "../models/neural_network_pos_specific_v2.pickle", "rb" ))

# Create pd dataframe of sequence information 
degron_pred = pd.read_csv("../data/degron_pred.csv", index_col=0)
# Select column of input sequences
input_seq = degron_pred.iloc[:, 10]

In [76]:
# Use compute_feature_matrix function to encode sequences
features = compute_feature_matrix(input_seq, 6, True)
condensed_features = features[:, 0:20] # Remove zeros at the end for bag of words prediction

In [77]:
# Use pre-trained model to predict sequence degredataion
pred_bow_wt = model_bag_of_words.predict_proba(condensed_features) # wild-type bag of words prediction
pred_ps_wt = model_pos_specific.predict_proba(features)            # wild-type position-specific prediction
drp_wt = pred_ps_wt-pred_bow_wt                                    # wild-type degron regulatory potential

In [78]:
# def mutate(seq, index, newchar):
#     char_list = list(seq)
#     char_list[index] = newchar
#     return "".join(char_list)

In [79]:
# Convert string to 2D list of characters

seqlist = [] # 2D list of all sequences, with each character as 1 entry 
for sequence in input_seq:
    seqlist.append(list(sequence))

### Mutation 1: Shift mutation
Shifts all the characters at a certin position down by 1 sequence, with the first sequence filled by the last sequence

In [80]:
seq_length = len(seqlist[0]) # 23
num_seq = len(seqlist)       # number of total sequences
shift = 1                    # distance to shift characters by

# List that contains the delta DRP for each position when a shift mutation occurs at that position
diff=[]

# Loop to iterate through all sequence positions
for p in range(seq_length):
    first = seqlist[0][pos]
    for i in range(num_seq-shift):
        #for j in range(seq_length) - iterate through positions
        seqlist[i][pos] = seqlist[i+shift][pos]
    seqlist[num_seq-shift][pos]=first
    
    mutated_seqs = []
    for i in range(num_seq):
        mutated_seqs.append("".join(seqlist[i])) 
        
    # Use compute_feature_matrix function to encode sequences
    features = compute_feature_matrix(pd.Series(mutated_seqs), 6, True)
    condensed_features = features[:, 0:20] # Remove zeros at the end for bag of words prediction

    # Use pre-trained model to predict sequence degredataion
    # Use pre-trained model to predict sequence degredataion
    pred_bow_mut = model_bag_of_words.predict_proba(condensed_features) # bag of words prediction
    pred_ps_mut = model_pos_specific.predict_proba(features)            # position-specific prediction
    drp_mut = pred_ps_mut-pred_bow_mut                                  # mutated degron regulatory potential
    
    # Calculate DRP difference in mutated and wild-type (normal) sequence and add this to a list
    diff.append(drp_wt-drp_mut)

In [82]:
average_delta = []
for i in diff:
    average_delta.append(np.average(i))

In [84]:
# # Plot histogram showing distribution of average DRP deltas

# sns.set(color_codes=True)
# x = np.random.normal(size=200)
# sns.distplot(diff)
# plt.show()

# sns.set(color_codes=True)
# x = np.random.normal(size=200)
# sns.distplot(average_delta, bins = 8)
# plt.show()

### Mutation 2: Random shuffle mutation (considering only Top & Bottom scores)
Shifts all the characters at a certain position using random shuffling, and then considers only the select top and bottom sequences

In [93]:
# Sort degron sequence information by regulatory potential in descending order (highest to lowest)
degron_pred_sorted = degron_pred.sort_values(by=['regulatory potential'], ascending=False)

In [98]:
# Select only the "sequence" and "potential" columns and format values into a list
degron_seq_scores = degron_pred_sorted[['Peptide amino acid sequence','regulatory potential']].values.tolist()

In [184]:
kmer_dict = {}

In [185]:
# Function description: 
#
# Cutoff -- number of sequences from the top + bottom to consider for motif analysis
# Motif length -- length of kmer motifs to be considered

def motif(cutoff = 50, motif_length = 2):
    top = degron_seq_scores[0:cutoff]
    bottom = degron_seq_scores[-cutoff:]
    kmers = []
    
    for row in top:
        sequence = row[0]
        for j in range(seq_length - motif_length + 1):
            kmers.append((sequence[j:(j+2)], row[1])) # add k-mer and corresponding score
    
    # Normalize drp scores to emphasize differences
    kmers = np.asarray(kmers) # Convert kmers to np array from a list
    x = np.asarray(list(map(float, kmers[:, 1])))
    normalized = (x-min(x))/(max(x)-min(x))
    kmers[:,1] = normalized
    
    # Convert list of kmers into 1 string
    kmer_str = " ".join(kmers[:, 0])
    kmer_list = list(kmers[:, 0])
    
    unique_kmers = [] 
  
    # Determine all unique kmers
    for i in kmer_list: 
        # checking for duplicacy 
        if i not in unique_kmers: 
            # insert value in str2 
            unique_kmers.append()  
    
    for i in range(0, len(unique_kmers)): 
        # count the frequency of each unique kmer and print 
        #print('Frequency of', unique_kmers[i], 'is :', kmer_list.count(unique_kmers[i]))
        kmer_dict.update({unique_kmers[i] : kmer_list.count(unique_kmers[i])})

In [ ]:
# Function description: 
#
# Cutoff -- number of sequences from the top + bottom to consider for motif analysis
# Motif length -- length of kmer motifs to be considered

def motif_with_weighting(cutoff = 50, motif_length = 2):
    top = degron_seq_scores[0:cutoff]
    bottom = degron_seq_scores[-cutoff:]
    kmers = []
    
    for row in top:
        sequence = row[0]
        for j in range(seq_length - motif_length + 1):
            kmers.append((sequence[j:(j+2)], row[1])) # add k-mer and corresponding score
    
    # Normalize drp scores to emphasize differences
    kmers = np.asarray(kmers) # Convert kmers to np array from a list
    x = np.asarray(list(map(float, kmers[:, 1])))
    normalized = (x-min(x))/(max(x)-min(x))
    kmers[:,1] = normalized
    
    # Convert list of kmers into 1 string
    kmer_str = " ".join(kmers[:, 0])
    kmer_list = list(kmers[:, 0])
    
    unique_kmers = [] 
  
    # Determine all unique kmers
    for i in kmer_list: 
        # checking for duplicacy 
        if i not in unique_kmers: 
            # insert value in str2 
            unique_kmers.append()  
    
    for i in range(0, len(unique_kmers)): 
        # count the frequency of each unique kmer and print 
        #print('Frequency of', unique_kmers[i], 'is :', kmer_list.count(unique_kmers[i]))
        kmer_dict.update({unique_kmers[i] : kmer_list.count(unique_kmers[i])})

In [186]:
motif()

In [190]:
sorted(kmer_dict.items(), key=lambda item: item[1])

[('IR', 1),
 ('RH', 1),
 ('FW', 1),
 ('PT', 1),
 ('FT', 1),
 ('VP', 1),
 ('AY', 1),
 ('CK', 1),
 ('CP', 1),
 ('AI', 1),
 ('QF', 1),
 ('QD', 1),
 ('DR', 1),
 ('RM', 1),
 ('CA', 1),
 ('NN', 1),
 ('YD', 1),
 ('TF', 1),
 ('MG', 1),
 ('YA', 1),
 ('NF', 1),
 ('IQ', 1),
 ('IF', 1),
 ('AP', 1),
 ('HV', 1),
 ('NM', 1),
 ('MP', 1),
 ('IV', 1),
 ('RD', 1),
 ('VW', 1),
 ('EM', 1),
 ('YW', 1),
 ('WL', 1),
 ('CE', 1),
 ('WT', 1),
 ('QN', 1),
 ('DY', 1),
 ('KW', 1),
 ('WY', 1),
 ('YY', 1),
 ('WA', 1),
 ('QM', 1),
 ('FG', 1),
 ('RK', 1),
 ('VD', 1),
 ('FH', 1),
 ('HY', 1),
 ('GW', 1),
 ('WR', 1),
 ('EY', 1),
 ('MR', 1),
 ('SY', 1),
 ('VH', 1),
 ('SM', 1),
 ('FF', 1),
 ('TI', 1),
 ('IP', 1),
 ('PM', 1),
 ('YH', 1),
 ('VM', 1),
 ('SN', 1),
 ('EP', 1),
 ('LH', 1),
 ('SF', 1),
 ('SW', 1),
 ('WV', 1),
 ('FL', 1),
 ('YE', 1),
 ('MF', 1),
 ('HR', 1),
 ('VV', 1),
 ('KR', 1),
 ('HE', 1),
 ('CD', 1),
 ('FC', 1),
 ('TM', 1),
 ('NS', 2),
 ('IL', 2),
 ('GH', 2),
 ('NR', 2),
 ('WD', 2),
 ('VA', 2),
 ('TY', 2),
 ('C

In [187]:
kmer_dict

{'AA': 13,
 'AC': 2,
 'AE': 10,
 'AF': 3,
 'AG': 5,
 'AH': 4,
 'AI': 1,
 'AK': 2,
 'AL': 7,
 'AM': 2,
 'AN': 4,
 'AP': 1,
 'AQ': 4,
 'AR': 5,
 'AS': 3,
 'AT': 10,
 'AV': 6,
 'AW': 2,
 'AY': 1,
 'CA': 1,
 'CD': 1,
 'CE': 1,
 'CK': 1,
 'CP': 1,
 'CQ': 2,
 'CR': 2,
 'CS': 3,
 'CT': 3,
 'CY': 2,
 'DA': 4,
 'DE': 4,
 'DF': 5,
 'DK': 5,
 'DL': 6,
 'DN': 4,
 'DP': 3,
 'DQ': 3,
 'DR': 1,
 'DS': 4,
 'DT': 3,
 'DY': 1,
 'EA': 6,
 'EC': 4,
 'ED': 3,
 'EE': 19,
 'EF': 3,
 'EG': 7,
 'EH': 2,
 'EI': 3,
 'EK': 11,
 'EL': 10,
 'EM': 1,
 'EN': 6,
 'EP': 1,
 'ER': 4,
 'ES': 8,
 'ET': 7,
 'EV': 10,
 'EY': 1,
 'FA': 3,
 'FC': 1,
 'FD': 2,
 'FE': 4,
 'FF': 1,
 'FG': 1,
 'FH': 1,
 'FI': 3,
 'FK': 2,
 'FL': 1,
 'FN': 3,
 'FP': 2,
 'FQ': 3,
 'FS': 5,
 'FT': 1,
 'FV': 2,
 'FW': 1,
 'GA': 10,
 'GD': 3,
 'GE': 5,
 'GF': 3,
 'GG': 11,
 'GH': 2,
 'GK': 3,
 'GL': 8,
 'GM': 2,
 'GN': 4,
 'GP': 2,
 'GQ': 5,
 'GR': 2,
 'GS': 7,
 'GT': 3,
 'GW': 1,
 'GY': 3,
 'HA': 2,
 'HE': 1,
 'HF': 2,
 'HG': 3,
 'HH': 5,
 'HK': 4,
 

In [165]:
def freq(str): 
  
    # break the string into list of words  
    str = str.split()          
    str2 = [] 
  
    # loop till string values present in list str 
    for i in str:              
  
        # checking for the duplicacy 
        if i not in str2: 
  
            # insert value in str2 
            str2.append(i)  
              
    for i in range(0, len(str2)): 
  
        # count the frequency of each word(present  
        # in str2) in str and print 
        print('Frequency of', str2[i], 'is :', str.count(str2[i]))     
  

In [189]:
kmer_dict['FN']

3

In [166]:
freq(" ".join(kmers[:, 0]))

Frequency of FN is : 3
Frequency of NS is : 2
Frequency of SE is : 9
Frequency of EA is : 6
Frequency of AH is : 4
Frequency of HL is : 5
Frequency of LL is : 10
Frequency of LP is : 11
Frequency of PI is : 3
Frequency of IL is : 2
Frequency of PK is : 4
Frequency of KD is : 6
Frequency of DK is : 5
Frequency of KK is : 4
Frequency of KE is : 7
Frequency of EV is : 10
Frequency of VE is : 4
Frequency of EI is : 3
Frequency of IR is : 1
Frequency of RE is : 3
Frequency of EE is : 19
Frequency of GH is : 2
Frequency of HS is : 7
Frequency of SH is : 8
Frequency of HH is : 5
Frequency of HG is : 3
Frequency of HP is : 3
Frequency of PS is : 5
Frequency of HQ is : 3
Frequency of QS is : 5
Frequency of SL is : 10
Frequency of PN is : 4
Frequency of NR is : 2
Frequency of RR is : 8
Frequency of RH is : 1
Frequency of VK is : 8
Frequency of KF is : 3
Frequency of FS is : 5
Frequency of EF is : 3
Frequency of FW is : 1
Frequency of WD is : 2
Frequency of DL is : 6
Frequency of LD is : 3
Freque